In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

# construct "table de fait"
histories = pd.read_csv("yearHistoryTemplate.csv")
weather = pd.read_csv("weather2017.csv")
histories['DATE'] = pd.to_datetime(histories['dep_time'], format = '%Y-%m-%d').dt.date
weather['DATE'] = pd.to_datetime(weather['DATE'], format = '%Y-%m-%d').dt.date
histories['precipitation'] = histories['DATE'].map(weather.set_index('DATE')['PRECIP_TOTAL_DAY_MM'])
histories['visibility'] = histories['DATE'].map(weather.set_index('DATE')['VISIBILITY_AVG_KM'])
histories['month'] = pd.to_datetime(histories['dep_time'], format = '%Y-%m-%d').dt.month
histories['hour'] = pd.to_datetime(histories['dep_time'], format = '%Y-%m-%d').dt.hour

,id,client_id,departure_id,arrival_id,vehicle_id,dep_time,arr_time,late_time,base_price,sup_price,DATE,precipitation,visibility,month,hour
0,1,1730,82,71,1772,2017-01-01 00:15:00,2017-01-01 00:19:00,0,1.84,0.0,2017-01-01,0.0,10.0,1,0
1,2,260,26,76,1441,2017-01-01 00:23:00,2017-01-01 00:27:00,0,3.04,0.0,2017-01-01,0.0,10.0,1,0
2,3,726,150,91,38,2017-01-01 02:04:00,2017-01-01 02:31:00,0,12.52,0.0,2017-01-01,0.0,10.0,1,2
3,4,1913,162,172,1791,2017-01-01 02:04:00,2017-01-01 02:18:00,0,8.36,0.0,2017-01-01,0.0,10.0,1,2
4,5,1994,38,41,1874,2017-01-01 03:18:00,2017-01-01 04:17:00,0,49.36,0.0,2017-01-01,0.0,10.0,1,3
5,6,791,145,25,307,2017-01-01 04:08:00,2017-01-01 05:07:00,0,42.51,0.0,2017-01-01,0.0,10.0,1,4
6,7,1110,57,135,1504,2017-01-01 05:38:00,2017-01-01 06:02:00,0,21.91,0.0,2017-01-01,0.0,10.0,1,5
7,8,484,41,7,524,2017-01-01 05:30:00,2017-01-01 06:30:00,0,34.75,0.0,2017-01-01,0.0,10.0,1,5
8,9,1737,54,38,1082,2017-01-01 05:26:00,2017-01-01 06:16:00,0,23.65,0.0,2017-01-01,0.0,10.0,1,5
9,10,1379,82,129,1067,2017-01-01 06:48:00,2017-01-01 06:57:00,0,7.97,0.0,2017-01-01,0.0,10.0,1,6


In [24]:
fait = histories.drop(['id','client_id', 'departure_id', 'arrival_id', 'vehicle_id',\
                       'dep_time', 'arr_time', 'base_price', 'sup_price', 'DATE'], axis=1)
fait.head(10)

,late_time,precipitation,visibility,month,hour
0,0,0.0,10.0,1,0
1,0,0.0,10.0,1,0
2,0,0.0,10.0,1,2
3,0,0.0,10.0,1,2
4,0,0.0,10.0,1,3
5,0,0.0,10.0,1,4
6,0,0.0,10.0,1,5
7,0,0.0,10.0,1,5
8,0,0.0,10.0,1,5
9,0,0.0,10.0,1,6


In [25]:
def rep_precip(x):
    if x < 3.0:
        return 1
    elif x >= 3. and x < 5.:
        return 2
    else :
        return 3

def rep_visib(x):
    if x < 5.:
        return 1
    elif x >= 5. and x < 8.:
        return 2
    else :
        return 3

def if_late(x):
    if x != 0:
        return 1
    else:
        return 0
def rep_month(x):
    if x is 7 or x is 8:
        return 1
    else:
        return 0
def rep_hour(x):
    if x in [7, 8, 12, 13, 19, 20]:
        return 1
    else:
        return 0
fait['precip'] = fait['precipitation'].apply(lambda x: rep_precip(x))
fait['visib'] = fait['visibility'].apply(lambda x: rep_visib(x))
fait['isLate'] = fait['late_time'].apply(lambda x: if_late(x))
fait['month'] = fait['month'].apply(lambda x: rep_month(x))
fait['hour'] = fait['hour'].apply(lambda x: rep_hour(x))
fait.drop(['late_time','visibility', 'precipitation'], axis=1, inplace=True)
fait.head(10)

,month,hour,precip,visib,isLate
0,0,0,1,3,0
1,0,0,1,3,0
2,0,0,1,3,0
3,0,0,1,3,0
4,0,0,1,3,0
5,0,0,1,3,0
6,0,0,1,3,0
7,0,0,1,3,0
8,0,0,1,3,0
9,0,0,1,3,0


In [23]:
# DONNO HOW TO TRANSFER IT
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [29]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

faitX = fait.drop('isLate', axis=1)
x_train, x_test, y_train, y_test = train_test_split\
(faitX, fait.isLate, test_size=0.33, random_state=0)
lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
score = metrics.accuracy_score(y_test, y_pred)
print("With logistic regression, we got " + str(score) + "%")

With logistic regression, we got 0.843895031395%


In [31]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_test)
score = metrics.accuracy_score(y_test, y_pred)
print("With bayes, we got " + str(score) + "%")

With bayes, we got 0.805743243243%


This ML is to predict if a cours is late. 
Features are precipitation, visibility month and hour. They are transfered in 0, 1, 2, three states.
Logistic regression shows better quality.